# NGIML Training (Local Runtime)
Minimal local-only notebook: uses local `prepared/` data and runs training without any Colab-specific steps.

In [ ]:
from pathlib import Path
import sys
import json
import dataclasses

REPO_ROOT = Path.cwd().resolve()
if not (REPO_ROOT / 'src').exists() and (REPO_ROOT.parent / 'src').exists():
    REPO_ROOT = REPO_ROOT.parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from tools.local_train_helpers import ensure_repo_on_syspath

ensure_repo_on_syspath(REPO_ROOT)

PREPARED_ROOT = REPO_ROOT / 'prepared'
OUTPUT_DIR = REPO_ROOT / 'runs' / 'ngiml_local'

print('Repo root:', REPO_ROOT)
print('Prepared root:', PREPARED_ROOT)
print('Output dir:', OUTPUT_DIR)

In [ ]:
from tools.local_train_helpers import build_manifest_from_prepared

MANIFEST_PATH = build_manifest_from_prepared(PREPARED_ROOT)
print('Manifest written to:', MANIFEST_PATH)

In [ ]:
from tools.local_train_helpers import build_local_training_config

# If False, IMD2020 views are automatically bumped to 2 to compensate.
BALANCE_SAMPLING = False
training_config = build_local_training_config(
    manifest_path=MANIFEST_PATH,
    output_dir=OUTPUT_DIR,
    balance_sampling=BALANCE_SAMPLING,
)

print(json.dumps(training_config, indent=2, default=lambda o: dataclasses.asdict(o) if dataclasses.is_dataclass(o) else str(o)))

In [ ]:
from importlib import reload
from tools import train_ngiml

reload(train_ngiml)
cfg = train_ngiml.TrainConfig(**training_config)
train_ngiml.run_training(cfg)